In [ ]:
%matplotlib inline

import animeface as af
import numpy as np

import PIL.Image
import os
import cv2 
import glob

import matplotlib.pyplot as plt
import time

In [ ]:
# Parameters
image_folder_path = os.getcwd() + '/images'
data_folder = 'data'
data_folder_path = os.getcwd() + '/' + data_folder
max_num = 140000

In [ ]:
# Count files
def countFile(file_path):
    n = 0
    for root, dirs, files in os.walk(file_path):
        n += len(files)
    return n

In [ ]:
# Start to process all the images. Following are the steps:
#   1. Detect anime faces
#   2. Crop and resize the detected faces
#   3. Save the new image
#   4. Delete the original image
image_type = [".jpg", ".png"]
image_index = 43662

while(countFile(data_folder_path) <= max_num):
    print("Waiting...")
    time.sleep(120)
    
    print("Start image processing...")
    if(os.path.isdir(image_folder_path)):
        image_list = []
        count = 0

        # Retrive all items from the folder
        for type in image_type:
            image_list.extend(glob.glob(image_folder_path + "/*" + type))

        # Start to process the images
        for image_path in image_list:
            print('Processing image ' + image_path[-10:])

            try:
                # Load the image
                image = PIL.Image.open(image_path)
                # Detect anime faces
                faces = af.detect(image)
                # Make a copy of the orginal image to draw face detections on
                image_cp = np.copy(image)

                for face_model in faces:
                    # Retrieve the bonding box positions
                    fp = face_model.face.pos
                    # Crop the face
                    face_cropped = image_cp[fp.y: fp.y + fp.height, fp.x: fp.x + fp.width]
                    # Resize the image to match 128 x 128 input
                    face_resized = cv2.resize(face_cropped, (128,128), interpolation = cv2.INTER_CUBIC)
                    # Store the resized image
                    if not os.path.exists(data_folder):
                        os.makedirs(data_folder)
                        
                    face_resized = PIL.Image.fromarray(face_resized)
                    face_resized.save(data_folder_path + '/img_' + str(image_index) + image_path[-4:])
                    # Increase the index
                    image_index += 1

                # Delete the original image
                os.remove(image_path)
                image.close()
                face_resized.close()
            except (IOError, OSError, ValueError):  ## if failed, report it back to the user ##
                try:
                    # Delete the original image
                    os.remove(image_path)
                except OSError:
                    pass
                pass
    
    print("End processing...")